Krzysztof Dobrucki\
Numer albumu 268507\
Informatyka Algorytmiczna\
Semestr Zimowy 2023/2024\
Politechnika Wrocławska\
29.10.2023r.

***
# Lista 1
## Obliczenia Naukowe
***

## Zadanie 1
Zadanie polegało na iteracyjnym wyznaczeniu kilku wartości, które charakteryzują arytmetykę.
### Epsilon maszynowy
Pierwszą z nich był macheps, czyli najmniejsza taka liczba epsilon, że $\epsilon$, że $ 1 + \epsilon > 1 $. Epsilon jest zatem odległością od 1 do najmniejszej liczby większej od 1 możliwej do zapisania w danej arytmetyce. Liczba 1 ma w zapisie dwójkowym mantysę wypełnioną zerami:

In [88]:
bitstring(Float16(1.0))

"0011110000000000"

z tego powodu można przewidywać, że $1 + \epsilon$ różni się od 1 w swoim najmniej znaczącym bicie. Rozpoczniemy proces sprawdzania epsilonów od wartości 1 i będziemy tworzyć kolejne poprzez dzielenie przez 2. W reprezentacji dwójkowej oznacza to przesuwanie jedynki w mantysie coraz bardziej w prawo. Będziemy postępować zgodnie z następującym algorytmem (iterative_epsilon), gdzie argumentem jest typ arytmetyki. Następnie porównamy wynik z wbudowaną funkcją jęzka Julia.

In [89]:
function iterative_epsilon(type)
    epsilon = type(1.0)
    while type(1.0 + 0.5*epsilon) > type(1.0)
        epsilon = type(0.5 * epsilon)
    end
    return epsilon
end

for i in [Float16, Float32, Float64]
    println(i)
    println(rpad("iterative_epsilon: ", 20), iterative_epsilon(i))
    println(rpad("in-build function: ", 20), eps(i), '\n')
end

Float16
iterative_epsilon:  0.000977
in-build function:  0.000977

Float32
iterative_epsilon:  1.1920929e-7
in-build function:  1.1920929e-7

Float64
iterative_epsilon:  2.220446049250313e-16
in-build function:  2.220446049250313e-16



Iteracyjnie wyniki są takie same jak te zwracane przez wbudowaną funkcję. Wartości do porównania z plikiem nagłówkowym **float.h** języka C zostały wzięte ze znalezionego artykułu w internecie.

FLT_EPSILON  = 1.19209e-07 (Float32)

DBL_EPSILON  = 2.22045e-16 (Float64)

LDBL_EPSILON = 1.0842e-19

Float16 nie ma odpowiednika, jednak pozostałe wartości są zbliżone.

### Epsilon a precyzja
Teraz zbadamy związek epsilonu maszynowego z precyzją. Precyzją arytmetyki, oznaczaną również jako $\epsilon$, definiuje się jako liczbę wyrażoną względem wzoru:

$$ \epsilon = \frac{1}{2}\beta^{1-t} $$

gdzie $\beta$ to baza reprezentacji (w naszym przypadku 2), a $t$ oznacza liczbę bitów w mantysie znormalizowanej do przedziału $[1/\beta,1)$. Dla typów Float16, Float32 i Float64 precyzje arytmetyki wynoszą odpowiednio:

Float16:  
$ 2^{-1}\cdot 2^{1-10} = 2^{-10} $  
Float32:  
$ 2^{-1}\cdot 2^{1-23} = 2^{-23} $  
Float64:  
$ 2^{-1}\cdot 2^{1-52} = 2^{-52} $

Teraz zweryfikujemy obliczone wartości precyzji dla odpowiadających im typów arytmetycznych.

In [90]:
println("Float16: ", Float16(2^-10))
println("Float32: ", Float32(2^-23))
println("Float64: ", Float64(2^-52))

Float16: 0.000977
Float32: 1.1920929e-7
Float64: 2.220446049250313e-16


Wartości precyzji, które otrzymaliśmy, są zgodne z wcześniejszymi obliczonymi epsilonami maszynowymi. Możemy zatem wnioskować, że w przypadku danej arytmetyki, wartość macheps jest równa jej precyzji (choć nie jest to wystarczający dowód).

### Liczba eta
Następną wartością do ustalenia jest liczba eta, co oznacza najmniejszą liczbę $\eta$ taka, że $\eta > 0$. Podobnie jak w przypadku epsilonu, intuicja opiera się na fakcie, że mantysa liczby 0 jest wypełniona zerami. W związku z tym, zastosowana metoda jest zupełnie analogiczna do powyższej.

In [91]:
function iterative_eta(type)
    eta = type(1.0)
    while type(0.5*eta) > 0
        eta = type(0.5*eta)
    end
    return eta
end

for i in [Float16, Float32, Float64]
    println(i)
    println(rpad("iterative_eta: ", 20), iterative_eta(i))
    println(rpad("in-build function: ", 20), nextfloat(i(0.0)), '\n')
end

Float16
iterative_eta:      6.0e-8
in-build function:  6.0e-8

Float32
iterative_eta:      1.0e-45
in-build function:  1.0e-45

Float64
iterative_eta:      5.0e-324
in-build function:  5.0e-324



Ponownie iteracyjnie wyliczona wartość jest ideantyczna z uzyskaną za pomocą wbudowanej funkcji.

### MIN<sub>sub</sub>
MIN<sub>sub</sub> to najmniejsza liczba w postaci nieznormalizowanej w danej arytmetyce, co oznacza, że jej znormalizowane reprezentowanie wymagałoby użycia wykładnika niższego, niż jest to dopuszczalne. Obliczamy ją zgodnie z następującym wzorem:

$$ \mathrm{MIN}_{sub} = 2^{1-t}\cdot2^{c_{min}} $$

gdzie $t$ oznacza liczbę cyfr mantysy (z zakresu $[1,2)$), a $c_{min}$ to minimalna możliwa cecha, wyznaczana ze wzoru:

$$ c_{min} = -2^{d-1}+2 $$

gdzie $d$ to liczba bitów przeznaczonych na zapis cechy. Dla typów w języku Julia obliczenia wyglądają następująco:

Dla Float16:  
$c_{min} = -2^{5-1}+2 = -14$  
MIN<sub>sub</sub> $= 2^{-10}\cdot2^{-14} = 2^{-24} $

Dla Float32:  
$c_{min} = -2^{8-1}+2 = -126$  
MIN<sub>sub</sub> $= 2^{-23}\cdot2^{-126} = 2^{-149} $

Dla Float64:  
$c_{min} = -2^{11-1}+2 = -1022$  
MIN<sub>sub</sub> $= 2^{-52}\cdot2^{-1022} = 2^{-1074} $

In [92]:
println("Float16: ", Float16(2^-24))
println("Float32: ", Float32(2^-149))
println("Float64: ", Float64(2^-1074))

Float16: 6.0e-8
Float32: 1.0e-45
Float64: 5.0e-324


Uzyskane wartości są zgodne z tymi z punktu **Liczba eta**

### MIN<sub>nor</sub>
Przeprowadźmy taki sam test dla funkcji floatmin() i porównajmy wyniki.

In [93]:
println("Float16: ", floatmin(Float16))
println("Float32: ", floatmin(Float32))
println("Float64: ", floatmin(Float64))

Float16: 6.104e-5
Float32: 1.1754944e-38
Float64: 2.2250738585072014e-308


Wyniki są większe od poprzednich.

In [94]:
bitstring(floatmin(Float16))

"0000010000000000"

Wynik wskazuje, że mamy do czynienia z formą znormalizowaną. Sprawdźmy związek wartości floatmin() z MIN<sub>nor</sub> (najmniejszą znormalizowaną wartością możliwą do zapisania), która wyrażana jest wzorem:
$$\mathrm{MIN}_{nor} = 2^{c_{min}}$$

gdzie $c_{min}$ jest takie samo jak dla MIN<sub>sub</sub>.

Float16:  
MIN<sub>nor</sub> $= 2^{-14} $

Float32:  
MIN<sub>nor</sub> $= 2^{-126} $

Float64:  
MIN<sub>nor</sub> $= 2^{-1022} $

In [95]:
println("Float16: ", Float16(2^-14))
println("Float32: ", Float32(2^-126))
println("Float64: ", Float64(2^-1022))

Float16: 6.104e-5
Float32: 1.1754944e-38
Float64: 2.2250738585072014e-308


Wartości MIN<sub>nor</sub> pokrywają się z floatmin().

### Największa liczba możliwa do wyrażenia
Ostatnią poszukiwaną wartością jest MAX, czyli największa możliwa liczba, jaką można wyrazić w danej arytmetyce. Intuicja podpowiada, że będzie to liczba, której wykładnik ma maksymalną dopuszczalną wartość, a mantysa składa się wyłącznie z jedynek.

In [96]:
function iterative_max(type)
    max = type(1.0)
    while !isinf(2*max)
        max *= 2
    end
    max *= (type(2.0) - eps(type))
    return max
end

for i in [Float16, Float32, Float64]
    println(i)
    println(rpad("iterative_max: ", 25), iterative_max(i))
    println(rpad("in-build function: ", 25), floatmax(i), '\n')
end

Float16
iterative_max:           6.55e4
in-build function:       6.55e4

Float32
iterative_max:           3.4028235e38
in-build function:       3.4028235e38

Float64
iterative_max:           1.7976931348623157e308
in-build function:       1.7976931348623157e308



Wyniki ponownie są takie same, czyli nasze iteracyjne podejście jest poprawne. Sprawdźmy jeszcze wartości dla języka C z pliku **float.h**:

FLT_MAX  = 3.40282e+38 (Float32)

DBL_MAX  = 1.79769e+308 (Float64)

LDBL_MAX = 1.18973e+4932

Float16 nie ma odpowiednika, jednak pozostałe wartości są zbliżone.

### Wnioski

Standard IEEE-754 narzuca pewne ograniczenia na dokładność reprezentacji liczb zmiennoprzecinkowych (jak każdy system oparty na maszynach liczących). Wzrost liczby bitów reprezentujących liczbę przekłada się na wzrost precyzji. W miarę jak liczba bitów rośnie, zmniejsza się najmniejsza możliwa do wyrażenia dodatnia liczba ($\eta$) oraz najmniejsza liczba większa od 1 ($1+\epsilon$), a jednocześnie znacząco rośnie maksymalna liczba możliwa do wyrażenia.
***

## Zadanie 2
Cel to eksperymentalne sprawdzenie prawidłowości wzoru Kahana na epsilon maszynowy:

$$ 3\cdot\left(\frac{4}{3}-1\right)-1 $$

Możemy porównać wyniki z tymi zwracanymi w podpunkcie **Epsilon maszynowy**.

In [97]:
function kahan_epsilon(type)
    type(3.0) * (type(4.0/3.0) - type(1.0)) - type(1.0)
end

for i in [Float16, Float32, Float64]
    println(i)
    println(rpad("kahan_epsilon: ", 20), kahan_epsilon(i))
    println(rpad("in-build functio: ", 20), eps(i), '\n')
end

Float16
kahan_epsilon:      -0.000977
in-build functio:   0.000977

Float32
kahan_epsilon:      1.1920929e-7
in-build functio:   1.1920929e-7

Float64
kahan_epsilon:      -2.220446049250313e-16
in-build functio:   2.220446049250313e-16



Z dokładnością do znaku wzór okazał się poprawny dla sprawdzanych arytmetyk.
***

## Zadanie 3

Eksperymentalnie zweryfikujemy, że w arytmetyce Float64 liczby z przedziału $[1,2]$ są równomiernie rozmieszczone z krokiem $\delta = 2^{-52}$. Ze względu na wielkość obliczeń sprawdzania wszystkich dostępnych liczb, skupimy się na wartościach wokół końców tego przedziału:

In [98]:
function check_values(start, delta, func)
    real = start
    for i in 1:1000
        start += delta
        real = func(real)
        if start != real
            return false
        end
    end
    return true
end

delta = 2.0^-52
if (check_values(1.0, delta, nextfloat) && check_values(2.0, -delta, prevfloat))
    println("For a given interval, the numbers are evenly distributed.")
else
    println("For a given interval, the numbers are NOT evenly distributed.")
end

For a given interval, the numbers are evenly distributed.


Sprawdźmy czy poza tym przedziałem wynik będzie taki sam:

In [99]:
if (1.0 - delta == prevfloat(1.0))
    println("True")
else
    println("False")
end

False


In [100]:
println(bitstring(prevfloat(1.0)))
println(bitstring(1.0))
println(bitstring(nextfloat(1.0)))

0011111111101111111111111111111111111111111111111111111111111111
0011111111110000000000000000000000000000000000000000000000000000
0011111111110000000000000000000000000000000000000000000000000001


Jeśli spróbujemy zrozumieć jak działa reprezentacja liczb w komputerach, czyli maszynach binarnych, możemy dojśc do wniosku, że taki wynik jest spodziewany. Przy przekroczeniu kolejnej potęgi dwójki potrzebujemy więcej bitów, aby zapisać części całkowite, co zmniejsza "miejsce" dla części ułamkowej.

Poniższy kod udowadnia to stwierdzenie:

In [101]:
exponent = parse(Int, bitstring(0.5)[2:12], base=2) 
real_exp = exponent - 1023 - 52
delta = 2.0^real_exp

if (check_values(0.5, delta, nextfloat) && check_values(1.0, -delta, prevfloat))
    println("[0.5; 1.0] = $delta = 2^$real_exp")
end

[0.5; 1.0] = 1.1102230246251565e-16 = 2^-53


In [102]:
exponent = parse(Int, bitstring(2.0)[2:12], base=2)
real_exp = exponent - 1023 - 52
delta = 2.0^real_exp

if (check_values(2.0, delta, nextfloat) && check_values(4.0, -delta, prevfloat))
    println("[2.0; 4.0] = $delta = 2^$real_exp")
end

[2.0; 4.0] = 4.440892098500626e-16 = 2^-51


### Wnioski
Liczby w przedziałach między kolejnymi potęgami liczby 2 są równomiernie rozmieszczone. Co istotne, ze względu na ograniczoną liczbę możliwych wartości mantysy, każdy taki przedział zawiera dokładnie taką samą liczbę liczb. W miarę jak odległości między kolejnymi potęgami dwójki rosną, krok $\delta$ na przedziałach między nimi musi również rosnąć.
***

## Zadanie 4

Należy znaleźć najmniejszą liczbę $x \in (1,2)$ typu Float64, dla której $x \cdot \frac{1}{x} \neq 1$. Zastosujemy więc najprostszą metodę. Będziemy przeglądać kolejne liczby od 1 w górę, aż do momentu, w którym warunek ten zostanie spełniony:

In [103]:
current = 1.0
while nextfloat(current) * (1.0/nextfloat(current)) == 1.0
    current = nextfloat(current)
end
println("The smallest number found is: $current")

The smallest number found is: 1.0000000572289969


### Wnioski
Ograniczenia sprzętowe w liczbie bitów reprezentujących daną liczbę powoduje czasami błędy.
***

## Zadanie 5
Celem zadania jest zaimplementowanie czterech różnych strategii obliczania iloczynu skalarnego dwóch wektorów $x$ i $y$, a następnie porównanie uzyskanych wyników.  

1. "w przód" - $\sum_{i=1}^n x[i]\cdot y[i]$:

In [104]:
function forward(x, y, type)
    sum = type(0.0)
    for i in 1:length(x)
        sum += x[i] * y[i]
    end 
    return sum
end

forward (generic function with 1 method)

2. "w tył" - $\sum_{i=n}^1 x[i]\cdot y[i]$:

In [105]:
function backward(x, y, type)
    sum = type(0.0)
    for i in length(x):-1:1
        sum += x[i] * y[i]
    end 
    return sum
end

backward (generic function with 1 method)

3. "od największego do najmniejszego" - dodaj dodatnie liczby w porządku od największego
do najmniejszego, dodaj ujemne liczby w porządku od najmniejszego do największego,
a następnie daj do siebie obliczone sumy częściowe

In [106]:
function descending_order(x, y)
    p = x .* y
    sum_pos = sum(sort(filter(a -> a>0, p), rev=true))
    sum_neg = sum(sort(filter(a -> a<0, p)))
    return sum_pos+sum_neg
end

descending_order (generic function with 1 method)

4. "od najmniejszego do największego" - przeciwnie do punktu 3.

In [107]:
function ascending_order(x, y)
    p = x .* y
    sum_pos = sum(sort(filter(a -> a>0, p)))
    sum_neg = sum(sort(filter(a -> a<0, p), rev=true))
    return sum_pos+sum_neg
end

ascending_order (generic function with 1 method)

Porównamy teraz powyższe sposoby obliczania iloczynu skalarnego dla danych z zadania z faktyczną wartością czyli $−1.00657107000000 \cdot 10^{−11}$.

In [108]:
x = [2.718281828, -3.141592654, 1.414213562, 0.5772156649, 0.3010299957]
y = [1486.2497, 878366.9879, -22.37492, 4773714.647, 0.000185049]

for i in [Float32, Float64]
    a = Array{i,1}(x)
    b = Array{i,1}(y)
    println(i)
    println(rpad("Precise value: ", 20), "-1.00657107000000e-11")
    println(rpad("Forward: ", 20), forward(a, b, i))
    println(rpad("Backward: ", 20), backward(a, b, i))
    println(rpad("Descending_order: ", 20), descending_order(a, b))
    println(rpad("Ascending_order: ", 20), ascending_order(a, b), '\n')
end

Float32
Precise value:      -1.00657107000000e-11
Forward:            -0.4999443
Backward:           -0.4543457


Descending_order:   -0.5
Ascending_order:    -0.5

Float64
Precise value:      -1.00657107000000e-11
Forward:            1.0251881368296672e-10
Backward:           -1.5643308870494366e-10


Descending_order:   0.0
Ascending_order:    0.0



### Wnioski
Jak widać uzyskaliśy różne wyniki. Na błędy w obliczeniach wpływa typ arytmetyki i kolejność wykonywania działań.
***

## Zadanie 6
Dane są dwie funkcje $f$ i $g$ dane wzorami:
$$ f(x) = \sqrt{x^2+1}-1 $$  
$$ g(x) = x^2/(\sqrt{x^2+1}+1) $$
Warto zauważyć, że: 
$$f = g$$
sprawdźmy, jakie wyniki daszą obie feunkcje dla:
$$x = 8^{-k}; k = 1, 2, 3, \dotso$$

In [109]:
function f(x)
    sqrt(x^2 + 1.) - 1.
end

function g(x)
    x^2 / (sqrt(x^2 + 1.) + 1.)
end

println(rpad("x", 8), rpad("|", 2), rpad("f(x)", 25), rpad("|", 2), rpad("g(x)", 30))
println("-"^8, "+", "-"^26, "+", "-"^26)
for i in 1:20
    println(rpad("8^-$i", 8), rpad("|", 2), rpad(f(8.0^-i), 25), rpad("|", 2), rpad(g(8.0^-i), 30))
end

x       | f(x)                     | g(x)                          
--------+--------------------------+--------------------------
8^-1    | 0.0077822185373186414    | 0.0077822185373187065         
8^-2    | 0.00012206286282867573   | 0.00012206286282875901        
8^-3    | 1.9073468138230965e-6    | 1.907346813826566e-6          
8^-4    | 2.9802321943606103e-8    | 2.9802321943606116e-8         
8^-5    | 4.656612873077393e-10    | 4.6566128719931904e-10        
8^-6    | 7.275957614183426e-12    | 7.275957614156956e-12         
8^-7    | 1.1368683772161603e-13   | 1.1368683772160957e-13        
8^-8    | 1.7763568394002505e-15   | 1.7763568394002489e-15        
8^-9    | 0.0                      | 2.7755575615628914e-17        
8^-10   | 0.0                      | 4.336808689942018e-19         
8^-11   | 0.0                      | 6.776263578034403e-21         
8^-12   | 0.0                      | 1.0587911840678754e-22        
8^-13   | 0.0                      | 1.65436122510605

### Wnioski
Matematycznie funkcje są sobie równe, jednak problemem jest odejmowanie małych liczb, więc funkcja $g$ daje bliższe prawdy wyniki. Wartości $x$ spadają do zera ponieważ dla bardzo małych $x$ mamy $\sqrt{x^2 + 1} \approx 1$, czyli odejmowanie małych liczb.
Jeśli chcemy uzyskać wyniki jak najbliższe prawdy powinniśmy przekształcić odpowiednio wzory, zamiast "ślepo" wrzucać do programu.
***

## Zadanie 7
W tym zadaniu skorzystamy ze wzoru na pochodną funkcji:
$$ f'(x) \approx \tilde{f}(x) = \frac{f(x+h)-f(x)}{h} $$  
Badana fukcja:
$$f(x) = sin(x) + cos(3x)$$
Pochodna badanej fukcji:
$$f'(x) = cos(x) + 3sin(3x)$$
Cel to porównanie wyników z dokładnymi wartościami.

In [110]:
println("f(1.0)  = ", (sin(1.0) + cos(3*1.0)))
println("f'(1.0) = ", (cos(1.0) - 3 * sin(3*1.0)))

f(1.0)  = -0.1485215117925489
f'(1.0) = 0.11694228168853815


In [111]:
function f(x)
    return sin(x) + cos(3x)
end

function real_df(x)
    return cos(x) - 3sin(3x)
end

function approx_df(x, h)
    return (f(x + h) - f(x)) / h
end

println(rpad("h", 8), rpad("|", 2), rpad("approx_value", 25), rpad("|", 2), rpad("error_value", 30))
println("-"^8, "+", "-"^26, "+", "-"^26)
for i in 1:54 
    println(rpad("2^-$i", 8), rpad("|", 2), rpad(Float64(approx_df(1.0, 2.0^-i)), 25), rpad("|", 2), rpad(abs(Float64(approx_df(1.0, 2.0^-i)-real_df(1.0))), 30))
end

h       | approx_value             | error_value                   
--------+--------------------------+--------------------------


2^-1    | 1.8704413979316472       | 1.753499116243109             
2^-2    | 1.1077870952342974       | 0.9908448135457593            
2^-3    | 0.6232412792975817       | 0.5062989976090435            
2^-4    | 0.3704000662035192       | 0.253457784514981             
2^-5    | 0.24344307439754687      | 0.1265007927090087            
2^-6    | 0.18009756330732785      | 0.0631552816187897            
2^-7    | 0.1484913953710958       | 0.03154911368255764           
2^-8    | 0.1327091142805159       | 0.015766832591977753          
2^-9    | 0.1248236929407085       | 0.007881411252170345          
2^-10   | 0.12088247681106168      | 0.0039401951225235265         
2^-11   | 0.11891225046883847      | 0.001969968780300313          
2^-12   | 0.11792723373901026      | 0.0009849520504721099         
2^-13   | 0.11743474961076572      | 0.0004924679222275685         
2^-14   | 0.11718851362093119      | 0.0002462319323930373         
2^-15   | 0.11706539714577957      | 0.000123115

Najdokładniejsze przybliżenie otrzymujemy, gdy $h$ wynosi $2^{-28}$. Dalsze zmniejszenie wartości $h$ skutkuje ponownym wzrostem błędu. Przyjrzyjmy się teraz wartościom wyrażenia $1+h$:

In [115]:
println(rpad("h", 8), rpad("|", 2), rpad("1+h", 22),  rpad("|", 2), rpad("f(1+h)", 22),  rpad("|", 2), rpad("f(1+h)-f(1)", 22))
println("-"^8, "+", "-"^23, "+", "-"^23,  "+", "-"^23)
for i in 1:54
    println(rpad("2^-$i", 8), rpad("|", 2), rpad(1.0+2.0^-i, 22), rpad("|", 2), rpad(f(1.0+2.0^-i), 22), rpad("|", 2), rpad(f(1.0+2.0^-i)-f(1.0), 22))
end

h       | 1+h                   | f(1+h)                | f(1+h)-f(1)           
--------+-----------------------+-----------------------+-----------------------
2^-1    | 1.5                   | 0.7866991871732747    | 0.9352206989658236    
2^-2    | 1.25                  | 0.12842526201602544   | 0.27694677380857435   
2^-3    | 1.125                 | -0.0706163518803512   | 0.07790515991219771   
2^-4    | 1.0625                | -0.12537150765482896  | 0.02315000413771995   
2^-5    | 1.03125               | -0.14091391571762557  | 0.0076075960749233396 
2^-6    | 1.015625              | -0.1457074873658719   | 0.0028140244266769976 
2^-7    | 1.0078125             | -0.14736142276621222  | 0.0011600890263366859 
2^-8    | 1.00390625            | -0.14800311681489065  | 0.0005183949776582653 
2^-9    | 1.001953125           | -0.1482777155172741   | 0.00024379627527482128
2^-10   | 1.0009765625          | -0.1484034624987881   | 0.00011804929376080242
2^-11   | 1.00048828125     

Widzimy, że dla $h \le 2^{-53}$ w arytmetyce Float64 mamy $1+h=1$, a więc $f(1+h)-f(1)=0$, stąd dalsze zmniejszanie $h$ nie spowoduje już zmiany w jakości przybliżenia. 

### Wnioski
Niekiedy nie jest możliwe proste przekształcenie wzoru w taki sposób, aby uniknąć utraty cyfr znaczących podczas operacji odejmowania. Błędy obliczeniowe, które występują, często są sprzeczne z intuicją matematyczną, która sugeruje, że używanie coraz mniejszych wartości $h$ prowadziłoby do poprawy dokładności przybliżenia.